In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
def true_1D_1(x): #True function for 1D_1 dy2/dx2 + dy/dx - 6y = 0; BC1: y(0)=2; BC2: dy/dx at (x=0) = -1;
    y = np.exp(2*x) + np.exp(-3*x)
    return y
    

In [3]:
loss_thresh = 0.005
label = "1D_SODE_swish"

x = np.linspace(0,2,100).reshape(-1,1)

bc1_x = x[0].reshape(-1,1)
bc1_y = true_1D_1(x[0]).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)
    

bc2_x = x[0].reshape(-1,1)
x_bc2_train = torch.from_numpy(bc2_x).float().to(device)
bc2_val = torch.tensor(-1.0,device=device)
bc2_val = bc2_val.view(1,1)

x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_true = true_1D_1(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

In [4]:
def colloc_pts(N_f,seed):
    #Collocation Points
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,y)
    x01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    
    x_coll_train = lb + (ub-lb)*sampling(N_f)
    x_coll_train = np.vstack((x_coll_train, bc1_x.reshape(-1,1))) # append training points to collocation points 

    return x_coll_train

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Sigmoid()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        self.beta = Parameter(torch.ones((layers[1],len(layers)-2)))
        self.beta.requiresGrad = True
    
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = z*self.activation(self.beta[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_BC2(self,x_bc2,bc2_val):
        g = x_bc2.clone()             
        g.requires_grad = True
        y = self.forward(g)    
            
        y_x = autograd.grad(y,g,torch.ones([x_bc2.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        dy_dx = y_x[:,[0]]
        
        bc2 = dy_dx
        
        loss_bc2= self.loss_function(bc2,bc2_val)

        return loss_bc2
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        y_xx = autograd.grad(y_x,g,torch.ones(x_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        dy2_d2x = y_xx[:,[0]]
        
        f = dy2_d2x + dy_dx - 6*y
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_bc2,bc2_val,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_bc2 = self.loss_BC2(x_bc2,bc2_val)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + loss_bc2 + loss_f
        
        return loss_val
          
    'test neural network'
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re

In [6]:
def train_step(x_coll,f_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [7]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [8]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    for i in range(max_iter):
        x_coll = torch.from_numpy(colloc_pts(N_f,i*11)).float().to(device)
        f_hat = torch.zeros(x_coll.shape[0],1).to(device)
        train_step(x_coll,f_hat)
        
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [9]:
max_reps = 10
max_iter = 100


N_f = 1000

train_loss_full = []
test_mse_full = []
test_re_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

for reps in range(max_reps):

    train_loss = []
    test_mse_loss = []
    test_re_loss =[]
    beta_val = []

    'Generate Training data'
    torch.manual_seed(reps*36)
     #Total number of collocation points 


    layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    PINN = Sequentialmodel(layers)
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())

    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.05, 
                              max_iter = 10, 
                              max_eval = 15, 
                              tolerance_grad = 1e-5, 
                              tolerance_change = 1e-5, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')


    train_model(max_iter,reps)


    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    beta_full.append(beta_val)    

    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "beta": beta_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 4.890845 Test MSE 383.40289852526513 Test RE 0.9981372842176024
1 Train Loss 4.890722 Test MSE 383.3609223797081 Test RE 0.9980826431299479
2 Train Loss 4.8907127 Test MSE 383.3565934788127 Test RE 0.9980770079532875
3 Train Loss 4.890395 Test MSE 383.2978703159622 Test RE 0.99800056

98 Train Loss 2.3809536 Test MSE 383.2641018016369 Test RE 0.9979565986701934
99 Train Loss 2.3809536 Test MSE 383.2641018016369 Test RE 0.9979565986701934
Training time: 19.89
Training time: 19.89
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 Train Loss 4.8909874 Test MSE 383.4216693875317 Test RE 0.9981617176116149
1 Train Loss 4.8907213

93 Train Loss 0.010255459 Test MSE 0.003047264005367677 Test RE 0.0028139580801271567
94 Train Loss 0.010247922 Test MSE 0.0025417067264031016 Test RE 0.002569953714522452
95 Train Loss 0.010243212 Test MSE 0.002199275859903758 Test RE 0.002390575345178924
96 Train Loss 0.010238941 Test MSE 0.002052469367461802 Test RE 0.0023094093755074876
97 Train Loss 0.010211411 Test MSE 0.0009732297755588677 Test RE 0.0015902678399845375
98 Train Loss 0.010077819 Test MSE 0.00032580339029196254 Test RE 0.000920111337692056
99 Train Loss 0.008404539 Test MSE 0.0006695298929869886 Test RE 0.0013190084219442525
Training time: 112.68
Training time: 112.68
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(

84 Train Loss 0.00097578764 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
85 Train Loss 0.00097578776 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
86 Train Loss 0.00097578764 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
87 Train Loss 0.00097578776 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
88 Train Loss 0.00097578764 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
89 Train Loss 0.0009757877 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
90 Train Loss 0.00097578776 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
91 Train Loss 0.0009757877 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
92 Train Loss 0.0009757877 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
93 Train Loss 0.0009757877 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
94 Train Loss 0.0009757877 Test MSE 5.789761871240155e-07 Test RE 3.87876022505774e-05
95 Train Loss 0.0009757877 Test MSE 5

75 Train Loss 0.0044483184 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
76 Train Loss 0.004448319 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
77 Train Loss 0.0044483184 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
78 Train Loss 0.004448319 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
79 Train Loss 0.004448319 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
80 Train Loss 0.0044483184 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
81 Train Loss 0.0044483184 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
82 Train Loss 0.004448319 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
83 Train Loss 0.004448319 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
84 Train Loss 0.004448319 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
85 Train Loss 0.0044483184 Test MSE 0.00017369225813253182 Test RE 0.0006718198215152721
86 Train Loss 0.004448319 T

...

56 Train Loss 0.0005762043 Test MSE 0.005355510629142768 Test RE 0.003730465477797664
57 Train Loss 0.00056843925 Test MSE 0.004066079057353044 Test RE 0.0032505024053166246
58 Train Loss 0.0005595529 Test MSE 0.002755641746883935 Test RE 0.002675925143531178
59 Train Loss 0.000554244 Test MSE 0.001979077208148797 Test RE 0.0022677436064349626
60 Train Loss 0.0005499283 Test MSE 0.0014646582234984556 Test RE 0.0019508807782118065
61 Train Loss 0.0005476577 Test MSE 0.001323345186205077 Test RE 0.001854381788368499
62 Train Loss 0.0005390431 Test MSE 0.0007692810160909873 Test RE 0.001413855710163247
63 Train Loss 0.00051586743 Test MSE 8.025198824381064e-05 Test RE 0.00045665738244346125
64 Train Loss 0.0005065315 Test MSE 1.4121313437369476e-05 Test RE 0.00019155792715675306
65 Train Loss 0.0005024123 Test MSE 5.666015229210495e-06 Test RE 0.00012133929052324579
66 Train Loss 0.0004932632 Test MSE 4.909829857140947e-07 Test RE 3.571871313192124e-05
67 Train Loss 0.00048573097 Test MSE

46 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
47 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
48 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
49 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
50 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
51 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
52 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
53 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
54 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
55 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
56 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
57 Train Loss 2.3751335 Test MSE 382.55815458350475 Test RE 0.9970370899120643
58 Train Loss 2.3751335 Test MSE 382.55815458350475 

41 Train Loss 0.00013444903 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05
42 Train Loss 0.00013444903 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05
43 Train Loss 0.00013444904 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05
44 Train Loss 0.00013444904 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05
45 Train Loss 0.00013444903 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05
46 Train Loss 0.00013444903 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05
47 Train Loss 0.00013444904 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05
48 Train Loss 0.00013444904 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05
49 Train Loss 0.00013444903 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05
50 Train Loss 0.00013444904 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05
51 Train Loss 0.00013444903 Test MSE 1.3830505270935213e-07 Test RE 1.8957523620915268e-05

28 Train Loss 2.3794236 Test MSE 383.2516104315457 Test RE 0.9979403358024688
29 Train Loss 2.3794172 Test MSE 383.2597477275701 Test RE 0.9979509300082097
30 Train Loss 2.379409 Test MSE 383.27163157667127 Test RE 0.997966401769074
31 Train Loss 2.3793993 Test MSE 383.28474881849 Test RE 0.9979834790238695
32 Train Loss 2.3793914 Test MSE 383.2936149171868 Test RE 0.9979950215766082
33 Train Loss 2.379382 Test MSE 383.300445675698 Test RE 0.998003914279683
34 Train Loss 2.3793259 Test MSE 383.33395476983395 Test RE 0.9980475373391137
35 Train Loss 2.3792112 Test MSE 383.3704311508278 Test RE 0.9980950211267764
36 Train Loss 2.377277 Test MSE 382.52372715714154 Test RE 0.9969922258922065
37 Train Loss 2.3377469 Test MSE 372.0401481711278 Test RE 0.983235355953853
38 Train Loss 2.3260682 Test MSE 370.82504983376106 Test RE 0.9816283993447918
39 Train Loss 2.3244748 Test MSE 370.08066411381003 Test RE 0.9806426552124425
40 Train Loss 2.316585 Test MSE 366.12581807618335 Test RE 0.9753887

24 Train Loss 1.4370553 Test MSE 177.30505882682658 Test RE 0.6787707919565261
25 Train Loss 0.78797007 Test MSE 76.19737229370322 Test RE 0.4449718936123293
26 Train Loss 0.44500804 Test MSE 40.39305765651575 Test RE 0.32397831867254856
27 Train Loss 0.3970383 Test MSE 35.83317275396194 Test RE 0.3051442664928644
28 Train Loss 0.21816477 Test MSE 3.3353065834593703 Test RE 0.09309588029446135
29 Train Loss 0.11419695 Test MSE 0.6440383976435335 Test RE 0.04090896226695658
30 Train Loss 0.09541426 Test MSE 0.414024323298944 Test RE 0.032800125886693746
31 Train Loss 0.07729113 Test MSE 1.0403580751286432 Test RE 0.05199409265395396
32 Train Loss 0.066109985 Test MSE 1.7791552253062881 Test RE 0.06799382963726595
33 Train Loss 0.02398222 Test MSE 0.266106009913868 Test RE 0.026296014106878637
34 Train Loss 0.017295722 Test MSE 0.002631966749743081 Test RE 0.002615187198895757
35 Train Loss 0.01691852 Test MSE 0.0005289687828600046 Test RE 0.001172404843530289
36 Train Loss 0.016854722 T

In [10]:
import scipy.io as sio

In [11]:
for tune_reps in range(5):
    label = "1D_SODE_swish_tune"+str(tune_reps)+".mat" #WRONGLY SAVED AS STAN - DOESN'T MATTER
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: '1D_SODE_swish_tune0.mat'